Install tflite or tflite Interpreter (the supported architecture version, see wheels here: https://github.com/iCorv/tflite-runtime).

In [ ]:
!pip install tflite

In [ ]:
#import tensorflow as tf
import tflite_runtime.interpreter as tflite
#from tensorflow.keras.utils import load_img, img_to_array
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import numpy as np
np.set_printoptions(suppress=True,
   formatter={'float_kind':'{:f}'.format})

In [ ]:
def get_label_encoder():
    encoder = LabelEncoder()
    encoder.classes_ = np.load('./best_classes.npy')
    return encoder

In [ ]:
# Load TFLite model and allocate tensors.

TFLITE_MODEL="./vecchio_modello_nuovo_dataset_55fotoclasse_hue.tflite"
#TFLITE_MODEL="./nuovoresnet152.tflite"
interpreter = tflite.Interpreter(TFLITE_MODEL)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load image (only with tf instead of tf lite)
#img = load_img("./images/pikachu_peluche.jpg", target_size=(224, 224))
#plt.imshow(img)

# tflite version
img = Image.open("./images/eevee_1.jpg")
img = img.resize((224, 224), Image.ANTIALIAS)
img = np.asarray(img, dtype=np.float32)
#img = img_to_array(img, dtype=np.float32)
img /= 255
img = np.expand_dims(img, axis=0)
# NB if your model was exported from pytorch, use this
#img = np.transpose(img, [0, 3, 1, 2])
print(img.shape)

input_tensor = np.array(img, dtype=np.float32)
# Load the TFLite model and allocate tensors.
interpreter.set_tensor(input_details[0]['index'], input_tensor)
interpreter.invoke()

# Get output
output_data = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
label_encoder = get_label_encoder()

In [ ]:
# tflite_runtime.interpreter as tflite method
results = np.squeeze(output_data, axis=0)
top_k_idx = np.argsort(results)[-5:][::-1]
top_k_values = results[top_k_idx]
top_k_labels = label_encoder.inverse_transform(top_k_idx)
print (top_k_labels, top_k_values)

Alternative way: use tensorflow and make prediction in this way.

In [ ]:
import tensorflow as tf

In [ ]:
# Tensorflow method
label_encoder = pc.get_label_encoder()
(top_k_scores, top_k_idx) = tf.math.top_k(output_data, 1)
top_k_scores = np.squeeze(top_k_scores.numpy(), axis=0)
top_k_idx = np.squeeze(top_k_idx.numpy(), axis=0)
top_k_labels = label_encoder.inverse_transform(top_k_idx)
print (top_k_labels, top_k_scores)